# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [133]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [150]:
ws = Workspace.from_config()
experiment_name = 'Capstone'
project_folder = './pipeline-project'

experiment=Experiment(ws, experiment_name)
experiment

dataset = Dataset.get_by_name(ws, name='covid-19-measures-and-deaths')
df = dataset.to_pandas_dataframe()
train_data = df.iloc[:-49]
test_data = df.iloc[-50:]

You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


In [135]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "worker"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

# compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [136]:
from azureml.automl.core.forecasting_parameters import ForecastingParameters

forecasting_parameters = ForecastingParameters(time_column_name='week', 
                                               forecast_horizon=10,
                                               time_series_id_column_names=["country"],
                                               freq='W',
                                               target_lags='auto',
                                               target_rolling_window_size=20)

automl_config = AutoMLConfig(task='forecasting',                             
                             primary_metric='normalized_root_mean_squared_error',
                             blocked_models = ['ExtremeRandomTrees', 'AutoArima', 'Prophet'],       
                             experiment_timeout_hours=0.3,
                             training_data=dataset,
                             label_column_name='rate_14_day',
                             compute_target=compute_target,
                             enable_early_stopping=False,
                             n_cross_validations=3,                             
                             verbosity=logging.INFO,
                             forecasting_parameters=forecasting_parameters)                                                             

In [137]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

In [138]:
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

In [139]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="Covid-19-aftermath-2",
    workspace=ws,    
    steps=[automl_step])

In [140]:
remote_run = experiment.submit(pipeline)

Created step automl_module [4b55cd7b][a8a63811-5fc7-49af-904f-cd8c8809c637], (This step will run and generate new outputs)
Submitted PipelineRun 98aeb11b-29e0-44d2-baaa-86aef460932e
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/Capstone/runs/98aeb11b-29e0-44d2-baaa-86aef460932e?wsid=/subscriptions/f08c5f25-28be-4c21-993c-ad64d5c84d3a/resourcegroups/ML/workspaces/capstone


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [141]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [142]:
remote_run.wait_for_completion()

PipelineRunId: 98aeb11b-29e0-44d2-baaa-86aef460932e
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/Capstone/runs/98aeb11b-29e0-44d2-baaa-86aef460932e?wsid=/subscriptions/f08c5f25-28be-4c21-993c-ad64d5c84d3a/resourcegroups/ML/workspaces/capstone
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: 44a8bc42-8dba-453b-a4d4-1391a7425cf2
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/Capstone/runs/44a8bc42-8dba-453b-a4d4-1391a7425cf2?wsid=/subscriptions/f08c5f25-28be-4c21-993c-ad64d5c84d3a/resourcegroups/ML/workspaces/capstone
StepRun( automl_module ) Status: NotStarted
StepRun( automl_module ) Status: Running

StepRun(automl_module) Execution Summary
StepRun( automl_module ) Status: Finished
{'runId': '44a8bc42-8dba-453b-a4d4-1391a7425cf2', 'target': 'worker', 'status': 'Completed', 'startTimeUtc': '2021-02-03T19:04:08.039425Z', 'endTimeUtc': '2021-02-03T19:27:33.494855Z', 'properties': {'ContentSnapshotId': '5b8b2fab-314

'Finished'

In [143]:
metrics_output = remote_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

Downloaded azureml/44a8bc42-8dba-453b-a4d4-1391a7425cf2/metrics_data, 1 files out of an estimated total of 1


,44a8bc42-8dba-453b-a4d4-1391a7425cf2_9,44a8bc42-8dba-453b-a4d4-1391a7425cf2_11,44a8bc42-8dba-453b-a4d4-1391a7425cf2_15,44a8bc42-8dba-453b-a4d4-1391a7425cf2_6,44a8bc42-8dba-453b-a4d4-1391a7425cf2_10,44a8bc42-8dba-453b-a4d4-1391a7425cf2_8,44a8bc42-8dba-453b-a4d4-1391a7425cf2_7,44a8bc42-8dba-453b-a4d4-1391a7425cf2_14,44a8bc42-8dba-453b-a4d4-1391a7425cf2_16,44a8bc42-8dba-453b-a4d4-1391a7425cf2_12,44a8bc42-8dba-453b-a4d4-1391a7425cf2_13
spearman_correlation,[0.40353147592254274],[0.36351101684319076],[0.3896775082971899],[0.28077991111011946],[0.3048358525256347],[0.38947061467257615],[0.3853043429860778],[0.3969186943123762],[0.4414623126400239],[0.32622241755894743],[0.4047989932380913]
root_mean_squared_error,[100.0008986999376],[102.85826886741312],[100.1025082729213],[103.56747176154663],[102.97167160845909],[102.66444876059082],[100.01849365421896],[99.77442693579944],[99.46762963021926],[102.80286388987992],[102.41093849625473]
explained_variance,[0.08044031853338114],[0.051386602154377625],[0.07983895261695832],[0.02965851369904619],[0.038253903253707956],[0.048332843514884094],[0.06801796493876593],[0.07822348498718786],[0.09041277878617375],[0.0441851354753614],[0.057478210293567776]
mean_absolute_percentage_error,[81.13323208072512],[79.3949240741931],[80.60154085823875],[81.97730025297462],[79.35565045018737],[79.0198001685061],[80.93503985065186],[78.95978674983108],[77.6706240175627],[80.06304830772119],[78.18086639822509]
mean_absolute_error,[62.707741225283236],[65.37063839241785],[62.654052463437914],[66.42502452194424],[65.70967334187732],[65.13637987170641],[62.03281895336452],[62.00344833212335],[61.68936350626232],[65.8102917922793],[64.99353531458563]
normalized_mean_absolute_error,[0.45988185788633235],[0.47440617895770726],[0.4582604595083159],[0.509836519023357],[0.5017575824519348],[0.47436353971302836],[0.4507198248822942],[0.45452673371582436],[0.4479268076438143],[0.5045665355906679],[0.47228260548563417]
normalized_median_absolute_error,[0.45249666966976226],[0.4638839921251466],[0.4511382286110013],[0.4790497604507052],[0.4762497730553157],[0.46057272918146214],[0.43929444690352154],[0.44157241212581505],[0.43695375126128283],[0.47798966137565485],[0.46481152684894506]
normalized_root_mean_squared_error,[0.5218879436360401],[0.5364075059939419],[0.5221081167105387],[0.5737360216415555],[0.5629467913502152],[0.5348115958935868],[0.5146740382746966],[0.5216284438165718],[0.5121042950080438],[0.566121361771821],[0.5318678939796292]
median_absolute_error,[43.94179548773544],[45.63646570964942],[44.392064490902534],[45.9895283087185],[44.954282498413086],[45.347768938869024],[44.04806848155942],[42.36160897525254],[42.480998333972416],[44.5674317788232],[45.55476597745556]
normalized_root_mean_squared_log_error,[0.4155024234458735],[0.4399003217940766],[0.41722855449933854],[0.546390955999756],[0.4907821763693858],[0.43232099189707013],[0.41227978762265655],[0.4385229587706949],[0.3945609092104227],[0.498585803226455],[0.4260180112761713]


## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [144]:
# Retrieve best model from Pipeline Run
best_model_output = remote_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

Downloaded azureml/44a8bc42-8dba-453b-a4d4-1391a7425cf2/model_data, 1 files out of an estimated total of 1


In [145]:
import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

ForecastingPipelineWrapper(pipeline=Pipeline(memory=None,
                                             steps=[('timeseriestransformer',
                                                     TimeSeriesTransformer(featurization_config=None,
                                                                           pipeline_type=<TimeSeriesPipelineType.FULL: 1>)),
                                                    ('prefittedsoftvotingregressor',
                                                     PreFittedSoftVotingRegressor(estimators=[('7',
                                                                                               Pipeline(memory=None,
                                                                                                        steps=[('robustscaler',
                                                                                                                RobustScaler(copy=True,
                                                                       

In [217]:
# part of last data per country
forecast_context = train_data[train_data.country=='Netherlands'][-5:]
# repeat the last row 12 times
test_set = pd.DataFrame(np.repeat(forecast_context[-1:].values, 12, axis=0))
test_set.columns = forecast_context.columns

# make a daterange in the future
forecast_horizon = pd.date_range(pd.Timestamp(2021, 2, 4), pd.Timestamp(2021, 4, 30), freq="W")

X_forecast = test_set.reset_index()
X_forecast['week'] = forecast_horizon
X_forecast['index'] = forecast_horizon
X_forecast = X_forecast.set_index('index')

# test_labels = test_set.pop('rate_14_day').values
# forecast_context.pop('rate_14_day').values


In [220]:
test_labels = X_forecast['rate_14_day'].values
test_labels

array([67.5262697151971, 67.5262697151971, 67.5262697151971,
       67.5262697151971, 67.5262697151971, 67.5262697151971, nan, nan,
       nan, nan, nan, nan], dtype=object)

In [224]:
# add the forecast context to the horizon
# X_forecast = forecast_context.append(X_forecast)
test_labels = X_forecast['rate_14_day'].values
test_labels = test_labels.fill(np.NaN)

label_fcst, data_trans = best_model.forecast(X_forecast, y_pred=test_labels,ignore_data_errors=True)


In [200]:

label_fcst, data_trans = best_model.forecast(forecast_destination=pd.Timestamp(2021, 3, 4))

In [225]:
data_trans[['_automl_target_col']]

,,,_automl_target_col
week,country,origin,
2021-02-07,Netherlands,2021-01-31,111.54
2021-02-14,Netherlands,2021-01-31,111.54
2021-02-21,Netherlands,2021-01-31,111.54
2021-02-28,Netherlands,2021-01-31,111.54
2021-03-07,Netherlands,2021-01-31,111.54
2021-03-14,Netherlands,2021-01-31,111.54
2021-03-21,Netherlands,2021-01-31,111.54
2021-03-28,Netherlands,2021-01-31,111.54
2021-04-04,Netherlands,2021-01-31,94.47


In [229]:
best_model

ForecastingPipelineWrapper(pipeline=Pipeline(memory=None,
                                             steps=[('timeseriestransformer',
                                                     TimeSeriesTransformer(featurization_config=None,
                                                                           pipeline_type=<TimeSeriesPipelineType.FULL: 1>)),
                                                    ('prefittedsoftvotingregressor',
                                                     PreFittedSoftVotingRegressor(estimators=[('7',
                                                                                               Pipeline(memory=None,
                                                                                                        steps=[('robustscaler',
                                                                                                                RobustScaler(copy=True,
                                                                       

In [238]:
remote_run.upload_file("outputs/my_model.pickle", best_model_output._path_on_datastore)
automl_model = remote_run.register_model(model_path="outputs/my_model.pickle", model_name='Covid-19-deaths-forecaster', description='Forecast the number of deaths per 100.000 due to covid-19.')
print(automl_model.name, automl_model.version, sep = '\t')


Covid-19-deaths-forecaster	1


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [239]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

env = Environment.get(workspace, "AzureML-Minimal").clone(env_name)

for pip_package in ["scikit-learn"]:
    env.python.conda_dependencies.add_pip_package(pip_package)

inference_config = InferenceConfig(entry_script='run.py', environment=env)

AttributeError: 'Model' object has no attribute 'deploy_model'

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service